In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
seed = 2
from xgboost import XGBClassifier
import pickle

In [2]:
#train = pd.read_csv('./data/new_train.csv')
train = pd.read_csv('./data/train_sample.csv')

In [3]:
sample_size = int(1e4)
try :
    sample = train[train.target == 1].sample(n=sample_size/2, random_state=seed)
    sample = sample.append( train[train.target == 0].sample(n=sample_size/2, random_state=seed))
except ValueError :
    sample = train[train.target == 1]
    sample = sample.append( train[train.target == 0].sample(n=sample_size-21694, random_state=seed) )

sample = sample.sample(frac=1, random_state=seed).reset_index(drop=True)
#sample.to_csv('./data/train_sample.csv', index=False)
train = sample

In [4]:
target = train.target
train.drop(['target','id'], inplace=True, axis=1)
train.drop([ col for col in train.columns if col.startswith('ps_cont') ],axis=1, inplace=True)

In [5]:
with open('./data/OneHotEncoder.clf', 'rb') as f:
    encoders = pickle.load(f)

In [6]:
enc_train = None

for feature,encoder in zip(train.columns,encoders) :
    encoded = encoder.transform(train[feature].values.reshape(-1,1))
    if enc_train is None :
        enc_train = encoded
    else :
        enc_train = np.concatenate((enc_train, encoded), axis=1)

In [ ]:
train = sample
#del train
#del enc_train

In [ ]:
with open('./data/model.pkl','rb') as f :
    model = pickle.load(f)

In [13]:
pred = model.predict(x_test)

In [15]:
from sklearn.metrics import classification_report
print classification_report(y_true=y_test, y_pred=pred)

             precision    recall  f1-score   support

          0       0.57      0.63      0.60       511
          1       0.57      0.51      0.54       489

avg / total       0.57      0.57      0.57      1000



In [10]:
def split_data(X, Y, ratio=0.1):
    test_ids = np.random.randint(0, X.shape[0], int(X.shape[0] * ratio))
    x_test = X[test_ids]
    y_test = Y[test_ids]
    x_train = np.delete(X, test_ids, axis=0)
    y_train = Y.drop(test_ids)
    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = split_data(enc_train, target, .1)

In [12]:
model.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.4, eval_metric='error', gamma=0,
       learning_rate=0.2, max_delta_step=0, max_depth=1,
       min_child_weight=7, missing=None, n_estimators=100, n_jobs=4,
       nthread=None, objective='binary:logistic', random_state=2,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.4)

In [9]:
model = XGBClassifier(
	objective = 'binary:logistic',
	n_jobs = 4,
	random_state = seed,
	eval_metric = "error",
	#-------
	max_depth = 1,
	min_child_weight = 7,
	#----------
	gamma = 0,
	#----------
	subsample = 0.4,
	colsample_bytree = 0.4,
	#------------
	scale_pos_weight = 1,
	#-----------
	reg_alpha = 0,
	reg_lambda = 1,
	#------------
	learning_rate = 0.2

	#n_estimators = 1000,
	)

In [8]:
GS_params1 ={
	'max_depth':range(3,10,2),
	'min_child_weight':range(3,10,2)
}
GS_params2 ={
	'max_depth':[1,2,3],
    'min_child_weight':[7]
}

GS_params3 ={
	'gamma':[i/10.0 for i in range(0,5)]
}
GS_params4 = {
 'subsample':[i/10.0 for i in range(1,6)],
 'colsample_bytree':[i/10.0 for i in range(1,6)]
}
GS_params5 = {
 'subsample':[.35,.4,.45],
 'colsample_bytree':[.35,.4,.45]
}
GS_params6 = {
 'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100],
 'reg_lambda':[1e-5, 1e-2, 0.1, 1, 100]
}
GS_params7 = {
'reg_alpha':[0, 1e-7, 1e-5 ,1e-6],
 'reg_lambda':[1]
}

gsearch = GridSearchCV(model, param_grid = GS_params7,n_jobs=4,pre_dispatch=4, cv=3, verbose=50 )
gsearch.fit(enc_train, target)
print gsearch.cv_results_, '\n' ,gsearch.best_params_,'\n' ,gsearch.best_score_


Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] reg_alpha=0, reg_lambda=1 .......................................
[CV] reg_alpha=0, reg_lambda=1 .......................................
[CV] reg_alpha=0, reg_lambda=1 .......................................
[CV] reg_alpha=1e-07, reg_lambda=1 ...................................
[CV] ........ reg_alpha=0, reg_lambda=1, score=0.590582, total=  11.9s
[CV] reg_alpha=1e-07, reg_lambda=1 ...................................
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:   12.1s
[CV] ........ reg_alpha=0, reg_lambda=1, score=0.583083, total=  12.8s
[CV] ........ reg_alpha=0, reg_lambda=1, score=0.584334, total=  12.8s
[CV] reg_alpha=1e-07, reg_lambda=1 ...................................
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:   13.1s
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed:   13.2s
[CV] reg_alpha=1e-05, reg_lambda=1 ...................................
[CV] .... reg_alpha=1e-07, reg_lambda=1, score=

GridSearchCV(cv=3, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.4, eval_metric='error', gamma=0,
       learning_rate=0.2, max_delta_step=0, max_depth=1,
       min_child_weight=7, missing=None, n_estimators=100, n_jobs=4,
       nthread=None, objective='binary:logistic', random_state=2,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.4),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'reg_alpha': [0, 1e-07, 1e-05, 1e-06], 'reg_lambda': [1]},
       pre_dispatch=4, refit=True, return_train_score=True, scoring=None,
       verbose=50)

{'std_train_score': array([ 0.00133943,  0.00133943,  0.00133943,  0.00133943]), 'rank_test_score': array([1, 1, 1, 1], dtype=int32), 'mean_score_time': array([ 0.08148066,  0.06799142,  0.05314795,  0.03784609]), 'param_reg_lambda': masked_array(data = [1 1 1 1],
             mask = [False False False False],
       fill_value = ?)
, 'std_test_score': array([ 0.00328032,  0.00328032,  0.00328032,  0.00328032]), 'param_reg_alpha': masked_array(data = [0 1e-07 1e-05 1e-06],
             mask = [False False False False],
       fill_value = ?)
, 'split1_train_score': array([ 0.60231023,  0.60231023,  0.60231023,  0.60231023]), 'split0_test_score': array([ 0.59058188,  0.59058188,  0.59058188,  0.59058188]), 'mean_test_score': array([ 0.586,  0.586,  0.586,  0.586]), 'split2_train_score': array([ 0.60467906,  0.60467906,  0.60467906,  0.60467906]), 'split0_train_score': array([ 0.60546055,  0.60546055,  0.60546055,  0.60546055]), 'params': ({'reg_alpha': 0, 'reg_lambda': 1}, {'reg_alpha':

In [ ]:
with open('./data/model.pkl', 'wb') as f:
    pickle.dump(file=f, obj=model)